# Clusterization

## 1. Texts Segmentation

In [3]:
from clusterization import text_segmentation
new = text_segmentation("../../../Texts/text2_3.txt")
new[:3]

[[(0,
   ' А когда речь заходит о покупке авто на вторичном рынке, то ещё и опасное.'),
  (0, ' Ведь подержанный автомобиль – это всегда «кот в мешке».'),
  (1,
   ' Вопреки описанию и виду, можно «напороться» и на технически неисправную машину, и на битую-перебитую, а порой и на «криминальную».'),
  (1,
   ' Однако добрую часть вариантов можно отсеять ещё на этапе просмотра объявлений, нужно только быть внимательным.')],
 [[(0, ''),
   (1, ' Грамотно составляйте запрос поиска\n'),
   (1,
    'Для того чтобы не тратить время и силы, нужно максимально точно определиться с тем, что, собственно, необходимо приобрести и сколько на покупку имеется денег.')]],
 [(0, ' Если нужен седан, значит, выбираете только седан.'),
  (0,
   ' Если нужна механическая коробка, в условиях поиска ставите «только МКП».'),
  (1,
   ' В идеале, конечно, к моменту составления запроса нужно четко представлять, автомобиль какой марки и модели предполагается к покупке.'),
  (0,
   ' Нужен Рено Логан, значит, смотр

## 2. Experiments With Semantic

In [4]:
import os
import numpy as np
import time
import pickle
from scipy.linalg import block_diag


In [5]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../../model.bin', binary=True) 
model.init_sims(replace=True)

/home/ased/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
import sys
sys.path.append('../../')
import sem_analysis as sa

In [7]:
newest = list()
ind =0
for i in new:
    _ = list()
    for j in i:
        if type(j) == type(tuple()):
            item = (j[0], sa.tag_ud(j[1]))
        else:
            item = list()
            for k in j:
                item.append((k[0], sa.tag_ud(k[1])))
        _.append(item)
    ind+=1
    newest.append(_)

In [20]:
full_list = list()
for i in newest:
    cur_list = [j for j in i]
    if len(cur_list) >0 and type(cur_list[0]) != type(tuple()):
        _ = list()
        for j in cur_list:
            _ = _ + j
        cur_list = _
    full_list = full_list + (cur_list)

In [23]:
sentences = list()
for i in new:
    cur_list = i
    if len(cur_list) >0 and type(cur_list[0]) != type(tuple()):
        _ = list()
        for j in cur_list:
            _ = _ + j
        cur_list = _
    sentences = sentences + (cur_list)

In [17]:
def get_center(sentence, model = model):
    s = np.zeros(300)
    k = 0
    for i in sentence:
        if model.__contains__(i):
            s = s + model[i]
            k+=1
    return s/k
full_list_centers = [(i[0], get_center(i[1])) for i in full_list]

/home/ased/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in true_divide


In [24]:
def distance(a, b):
    return np.linalg.norm(a - b)

In [25]:
c_prev, c_next = None, None
eps = 0.9
res = list()
for _, j in enumerate(full_list_centers):
    if j[0] == 1:
        c_next = (_,j)
        break
for ind, i in enumerate(full_list_centers):
    if i[0] == 1:
        c_prev = (ind, i)
        res.append((ind, 0.))
        c_next = None
        for _, j in enumerate(full_list_centers[ind+1:]):
            if j[0] == 1:
                c_next = (ind+1+_,j)
                break
    else:
        if not c_prev is None:
            d_prev = distance(c_prev[1][1], i[1])
        else:
            d_prev = 1000
        if not c_next is None:
            d_next = distance(c_next[1][1], i[1])
        else:
            d_next = 1000
        if d_next > eps and d_prev > eps:
            res.append((None, None))
        elif d_next > d_prev: 
            res.append((c_prev[0], d_prev))
        else:
            res.append((c_next[0], d_next))

In [26]:
res

[(2, 0.4559460808929187),
 (2, 0.5466523485902612),
 (2, 0.0),
 (3, 0.0),
 (5, nan),
 (5, 0.0),
 (6, 0.0),
 (9, 0.4963257939978532),
 (6, 0.416295412819199),
 (9, 0.0),
 (9, 0.4910867621420883),
 (11, 0.0),
 (11, 0.4163209590439051),
 (11, 0.43258011507331107),
 (15, nan),
 (15, 0.0),
 (18, 0.43553505651044094),
 (18, 0.38972150611591655),
 (18, 0.0),
 (18, 0.4540353343193805),
 (18, 0.46168586706073517),
 (22, nan),
 (22, 0.0),
 (25, 0.3856013242573196),
 (25, 0.3836830112230355),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (29, nan),
 (29, 0.0),
 (31, 0.6600905685791763),
 (31, 0.0),
 (31, 0.4287097267277469),
 (31, 0.41713547505313164),
 (31, 0.32200362115451725),
 (31, 0.3898936600336726),
 (36, 0.0),
 (38, nan),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (41, 0.45248898161699247),
 (41, 0.5784668686281154),
 (41, 0.41698657269041506),
 (41, 0.5545150888283829),
 (41, 0.46515206730471825),
 (48, nan),
 (48, 0.0),
 (49, 0.0),
 (51, 0.4422508803412458),
 (51, 0.0),
 (51, 0.34897485774

In [30]:
prev = None
cur_text = ''
cur_item = None
list_texts = list()
TT = list()
for ind, i in enumerate(res):
    if i[0] == prev:
        cur_text = cur_text + sentences[ind][1]
        cur_item = cur_item + full_list_centers[ind][1]
        k += 1
    else:
        if not i[0] is None:
            prev = i[0]
            if not cur_item is None:
                print(cur_text, '\n')
                list_texts.append(cur_text)
                TT.append(cur_item / k)
            cur_text = sentences[ind][1]
            cur_item = full_list_centers[ind][1]
            k = 1
print(cur_text)
list_texts.append(cur_text)
TT.append(cur_item / k)

 А когда речь заходит о покупке авто на вторичном рынке, то ещё и опасное. Ведь подержанный автомобиль – это всегда «кот в мешке». Вопреки описанию и виду, можно «напороться» и на технически неисправную машину, и на битую-перебитую, а порой и на «криминальную». 

 Однако добрую часть вариантов можно отсеять ещё на этапе просмотра объявлений, нужно только быть внимательным. 

 Грамотно составляйте запрос поиска
 

Для того чтобы не тратить время и силы, нужно максимально точно определиться с тем, что, собственно, необходимо приобрести и сколько на покупку имеется денег. 

 Если нужен седан, значит, выбираете только седан. 

 Если нужна механическая коробка, в условиях поиска ставите «только МКП». 

 В идеале, конечно, к моменту составления запроса нужно четко представлять, автомобиль какой марки и модели предполагается к покупке. Нужен Рено Логан, значит, смотрим Рено Логан, а не Clio Symbol, Sandero и Polo Sedan до кучи. 

 Имейте в виду, что даже самый «замечательный» б/у автомобиль п

In [31]:
D = [(np.linalg.norm(i - TT[ind+1]), list_texts[ind], list_texts[ind+1]) for ind, i in enumerate(TT[:-1])]

In [32]:
D

[(0.39145511719774523,
  ' А когда речь заходит о покупке авто на вторичном рынке, то ещё и опасное. Ведь подержанный автомобиль – это всегда «кот в мешке». Вопреки описанию и виду, можно «напороться» и на технически неисправную машину, и на битую-перебитую, а порой и на «криминальную».',
  ' Однако добрую часть вариантов можно отсеять ещё на этапе просмотра объявлений, нужно только быть внимательным.'),
 (nan,
  ' Однако добрую часть вариантов можно отсеять ещё на этапе просмотра объявлений, нужно только быть внимательным.',
  ' Грамотно составляйте запрос поиска\n'),
 (nan,
  ' Грамотно составляйте запрос поиска\n',
  'Для того чтобы не тратить время и силы, нужно максимально точно определиться с тем, что, собственно, необходимо приобрести и сколько на покупку имеется денег.'),
 (0.5197614053842909,
  'Для того чтобы не тратить время и силы, нужно максимально точно определиться с тем, что, собственно, необходимо приобрести и сколько на покупку имеется денег.',
  ' Если нужен седан, з